### Importaciones

In [303]:
# Paquetes necesarios para la ejecución del notebook
import win32com.client
import os
from datetime import datetime
import pandas as pd
import shutil
from openpyxl import load_workbook
from openpyxl.workbook import Workbook
from copy import copy
from openpyxl.utils import get_column_letter

### Configuracion

In [304]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"
MAIL_ITEM_CODE = 43

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}
parse_locaciones = {
    '06 AYA EL PEDREGAL': 'El Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}
vendedores = {
    '06 AYA EL PEDREGAL': {
        '0000001013': 'AYALA CCAHUANA EDMUNDO',
        '0000001006': 'BARRAZA REVILLA FREDY',
        '0000001000': 'CARDENAS CHOQUECAJIA WHITNEY AWARD',
        '0000001001': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001007': 'CHINO QUISPE JORGE LUIS',
        '0000006014': 'EDWAR QUISPE CHINO',
        '0000006009': 'GALLARDO HUAYLLA RENATO LEUTERIO',
        '0000006003': 'GALLEGOS NUÑEZ JUAN FREDY',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001010': 'HUANCA MAMANI JACKSON',
        '0000006006': 'HUAYCHO TORRES ULBER',
        '0000006005': 'HUAYCHO TORRES ULBER',
        '0000001012': 'HUAYLLAZO HUACCHA CRISTHIAN PAOLO',
        '0000006012': 'NELSON RAUL CONZA CHARCA',
        '0000001002': 'PACHECO CONDORI ANDRES OSCAR',
        '0000001011': 'QUISPE CHINO WILIAN YURI',
        '0000001004': 'QUISPE HUAYLLA MARISOL S.',
        '0000006013': 'RONALD GONZALO HUILLCA MAMANI',
        '0000001003': 'TACO XESSPE KELY SOFIA',
        '0000001005': 'VALDERRAMA ELLIS JESSICA A.',
        '0000001008': 'VEND NVA RUTA 10',
        '0000001009': 'YUCRA JIMENEZ ANA LUZ',
        '0000001014': 'CHARA ROJAS GISELA SOLEDAD',
        '0000001015': 'BARRAZA REVILLA FREDY',
        '0000001019': 'BATALLANOS SANCA RODRIGO LEOPOLDO',
        '0000001016': 'PARI PUCHO FREDY OSWALDO',
        '0000001017': 'COAGUILLA MAMANI JOSE ALBERTO',
        '0000001018': 'ORTEGA MAMANI JORGE LUIS'
    },
    '38 AYA ATICO': {
        '0000001001': 'ALANYA RAMIREZ FERNANDO JOSUE',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001000': 'SAUL ANDRES VIÑA VIZCARDO'
    },
    '40 AYA CHALA': {
        '0000001004': 'CANALES AGUILAR HILBERTO',
        '0000001007': 'CHIPANA JURADO JHORS EDUARDO',
        '0000006010': 'DIONICIO DANIEL HUARCAYA SALAZAR',
        '0000005999': 'GESTOR VIRTUAL',
        '0000006009': 'GLOBER FELIPE JARA MAQUER',
        '0000006005': 'GONZALES CHURA MAGNO ALFREDO',
        '0000001000': 'HERRERA TAPIA EDWIN DONATO',
        '0000006002': 'JARA CARAZAS RODOLFO JOSUE.L',
        '0000006007': 'JUAN CARLOS ARIAS BENITES',
        '0000001003': 'MAMANI TINTAYA KRISTHOFER',
        '0000001006': 'MAMANI TINTAYA KRISTHOFER',
        '0000001005': 'QUISE CCAPA EVER',
        '0000001002': 'RODRIGUEZ JOSE ANTONIO',
        '0000001001': 'VENDEDOR RT M1'        
    },
    '88 AYA CAMANA': {
        '0000001007': 'AUCAHUAQUI REVILLA DANIEL HITLER',
        '0000001011': 'CARAZAS REZA LUIS ALBERTO',
        '0000001003': 'CONDORCHOA SIERRA NEMECIO',
        '0000006001': 'CONDORCHOA SIERRA NEMESIO JESUS',
        '0000006002': 'DE LA CRUZ CALCINA ELAR',
        '0000006010': 'EDILBERTO RAMIREZ LARICO',
        '0000005999': 'GESTOR VIRTUAL',
        '0000001002': 'HUAMANI RODRIGUEZ YONATAN ANYIMZAN',
        '0000006011': 'JAIME CHAVEZ CONDORI',
        '0000001005': 'LLERENA DE LA CRUZ RICARDO SNEIDER',
        '0000001004': 'MEDINA VELASQUEZ JAVIER ENRIQUE',
        '0000006014': 'MOISES RICHARD CONDORCHOA SIERRA',
        '0000001008': 'MOLLO YUPANQUI JOSE OMAR',
        '0000001001': 'NO APLICA VENDEDOR',
        '0000006008': 'QUISPE CCACHUCO FREDY',
        '0000001010': 'RAMOS MAMANI RUBEN',
        '0000006016': 'RENATO ELEUTERIO GALLARADO HUAYLLA',
        '0000001006': 'SACARI CHOQUEHUANCA WILSON',
        '0000001000': 'SALAZAR HUAMANI SAUL ANTONIO',
        '0000001012': 'VEND RT X1',
        '0000001009': 'VIZCARDO BUSTAMANTE ALBERTH FRANCK'
    }
}

# Listas
locaciones = ['06 AYA EL PEDREGAL', '38 AYA ATICO', '40 AYA CHALA', '88 AYA CAMANA']

# Variables de configuracion
root_address = r'C:\Informacion\rechazos' # Direcccion de carpeta raiz
test_address = r'\prueba'
backup_address = r'\backup'
transportista = {
    "mail_subject": "Reporte de ordenes de carga diario",         # Nombre del asunto de correo
    "local_file_name": "Cf_programadas_por_transportista.xlsx",   # Nombre del archivo local
    "mail_file_address": "",                                      # Direccion del archivo en el correo
    "local_sheet_name": "Programación",
    "mail_sheet_name": "Guias",
    "date": "Fecha",
    "table": "Tabla5"
}
ruta = {
    "mail_subject": "Venta perdida diaria por cliente y ruta - acum mes",   # Nombre del asunto de correo
    "local_file_name": "Cf_rech_por_ruta.xlsx",                             # Nombre del archivo local
    "mail_file_name": "",                                                   # Direccion del archivo en el correo
    "local_sheet_name": "Motivos_VP (clte)",
    "mail_sheet_name": "Motivos_VP (clte)",
    "date": "Día",
    "table": "Tabla2"
}

### Obtener correos de outlook

In [305]:
# Conectar a Outlook
#outlook_folder_code = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(6)
print("Tipo de folder: ", outlook_folder)

Tipo de folder:  Bandeja de entrada


### Guardar archivo de outlook

In [306]:
# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

def download_excel_file(mails, mail_subject):
    for mail in mails: # Recorrer todos los correos
        if mail.Class != MAIL_ITEM_CODE: # Asegurar que el correo sea un MailItem (otros pueden ser calendario, alertas, etc)
            continue
        if (mail_subject.lower() in mail.Subject.lower()) & (mail.Attachments.Count > 0):
            for attachment in mail.Attachments:
                if attachment.FileName.endswith((".xlsx", ".xls")):
                    file_name = attachment.FileName
                    file_address = os.path.join(root_address+test_address, file_name)
                    attachment.SaveAsFile(file_address)
                    return file_address            

transportista['mail_file_address'] = download_excel_file(mails, transportista['mail_subject'])
ruta['mail_file_address'] = download_excel_file(mails, ruta['mail_subject'])

print(transportista['mail_file_address'])
print(ruta['mail_file_address'])

C:\Informacion\rechazos\prueba\Reporte de Guias de Distribución por FechaV3_855_4894372674151982690.xlsx
C:\Informacion\rechazos\prueba\Venta perdida x Cliente y ruta diaria_725_605084156184953594.xlsx


### Actualizar archivos fuente

In [307]:
def clean_mail_file(df):
    # 1: Eliminar la primera columna
    df = df.dropna(axis=1, how='all')

    # 2: Tomar la fila 1 como nombres de columnas
    df.columns = df.iloc[1]

    # 3: Eliminar las dos primeras filas (la original de encabezado y la fila de nombres)
    df = df.iloc[2:].reset_index(drop=True)
    
    return df

def filter_mail_file_locations(df):
    # Conservar las filas con locaciones especificas
    df = df[df['Locación'].isin(locaciones)]

    return df

def customized_mail_file(df):
    # Personalizada 1: Eliminar ultima columna
    if 'Mesa Comercial' in df.columns:
        df = df.drop(columns=['Mesa Comercial'])

    return df

# Copia estilos de una fila anterior (por ejemplo, la última)
def copiar_formato_fila(origen_ws, destino_ws, fila_origen, fila_destino):
    for col in range(1, origen_ws.max_column + 1):
        celda_origen = origen_ws.cell(row=fila_origen, column=col)
        celda_destino = destino_ws.cell(row=fila_destino, column=col)

        # Copiar estilo, número, alineación, borde, etc.
        if celda_origen.has_style:
            celda_destino.font = copy(celda_origen.font)
            celda_destino.border = copy(celda_origen.border)
            celda_destino.fill = copy(celda_origen.fill)
            celda_destino.number_format = copy(celda_origen.number_format)
            celda_destino.protection = copy(celda_origen.protection)
            celda_destino.alignment = copy(celda_origen.alignment)

In [ ]:
def update_local_file(document, custom=False):
    # Leer datos del archivo de correo
    df_mail = pd.read_excel(document['mail_file_address'], sheet_name=document['mail_sheet_name'], header=None)

    # Direccion de archivo local
    local_file_address = os.path.join(root_address+test_address, document['local_file_name'])

    # Backup del archivo
    local_file_backup_address =  os.path.join(root_address+test_address+backup_address, document['local_file_name'])
    shutil.copy2(local_file_address, local_file_backup_address) # Crear el backup

    # Leer datos existentes del archivo local (sin tocar fórmulas)
    df_local = pd.read_excel(local_file_address, sheet_name=document['local_sheet_name'], dtype=str, engine='openpyxl')

    # Limpiar archivo de correo (dataframe)
    df_mail = clean_mail_file(df_mail)
    df_mail = filter_mail_file_locations(df_mail)
    df_mail = customized_mail_file(df_mail) if custom else df_mail

    # Convertir columna 'Fecha' a datetime en archivo recibido
    df_mail[document['date']+"2"] = pd.to_datetime(df_mail[document['date']], dayfirst=True, errors='coerce')

    # Fecha máxima en archivo local
    fecha_max_local = df_local[document['date']].max()

    # Filtrar solo filas con fecha mayor que la máxima del local
    df_mail = df_mail[df_mail[str(document['date']+"2")] > fecha_max_local].copy()

    # Eliminar columna date2
    df_mail = df_mail.drop(columns=[str(document['date']+"2")])

    # Ajustar columnas del archivo del correo para que tenga los nombres del archivo local por posición
    # Cortamos a la misma cantidad de columnas que tiene el correo
    df_local_same_columns = df_local.columns[:len(df_mail.columns)]

    # Reemplazar los nombres del DataFrame del correo con los nombres del local, por posición
    df_mail.columns = df_local_same_columns

    # Asegurar columnas del archivo local están en df_nuevas_filas (llenar con None si faltan)
    for col in df_local.columns:
        if col not in df_mail.columns:
            df_mail[col] = None

    print('Datos nuevos: ', len(df_mail))
    #print(df_mail.info())

    # Abrir el archivo local para escribir
    wb = load_workbook(local_file_address)
    ws = wb[document['local_sheet_name']]

    # Buscar la última fila con datos
    last_row = ws.max_row + 1
    print("Ultima fila: ", last_row) # transportista=11353, ruta=8286

    # Escribir las nuevas filas
    if len(df_mail) > 0:
        for _, row in df_mail.iterrows(): # Recorre cada fila
            copiar_formato_fila(ws, ws, last_row - 1, last_row)
            for idx, col in enumerate(df_local.columns, start=1): # Recorre cada columna
                celda = ws.cell(row=last_row, column=idx)  # Obtener celda actual
                valor = row[col]

                # Detectar si la columna es la fecha de la hoja
                if col == document['date']:
                    # Asegurarse que es datetime
                    # fecha_valor = pd.to_datetime(row[col], dayfirst=True, errors='coerce')
                    # celda.value = fecha_valor
                    celda.number_format = 'd\/m\/yy' # Formato visual en Excel
                    celda.value = datetime.strptime(str(valor), '%Y-%m-%d %H:%M:%S')

                # Detectar si la columna es 'Nombre Vendedor'
                if custom and (col == 'Nombre Vendedor'):
                    try:
                        celda.value = vendedores[row['Locación']][row['VendedorCod']]
                    except:
                        celda.value = "NO DEFINIDO" # O dejarlo vacío
                else:
                    # Celdas normales
                    celda.value = row[col]

            last_row += 1
    else:
        print("No hay datos nuevos (El archivo del correo esta desactualizado)")


    # Extender el rango del autofiltro si existe
    # col_count = ws.max_column
    # new_last_row = ws.max_row
    # filter_range = f"A1:{get_column_letter(col_count)}{new_last_row}"
    # ws.auto_filter.ref = filter_range

    # Extender tabla existente
    table_name = document["table"]
    found = False

    for tbl in ws.tables.values():
        if tbl.name == table_name:
            found = True
            # Obtener celda de inicio
            start_cell = tbl.ref.split(":")[0]
            start_col_letter = ''.join(filter(str.isalpha, start_cell))
            start_row = int(''.join(filter(str.isdigit, start_cell)))
            
            # Calcular nueva última celda (misma cantidad de columnas que la tabla original)
            num_cols = len(tbl.tableColumns)
            end_col_letter = get_column_letter(openpyxl.utils.column_index_from_string(start_col_letter) + num_cols - 1)
            end_row = ws.max_row
            new_ref = f"{start_col_letter}{start_row}:{end_col_letter}{end_row}"
            
            tbl.ref = new_ref
            ws.auto_filter.ref = new_ref
            print(f"✅ Tabla '{tbl.name}' actualizada a rango: {new_ref}")
            break

    if not found:
        print(f"⚠️ Tabla '{table_name}' no encontrada. No se actualizó rango.")

    # Guardar cambios
    wb.save(local_file_address)

update_local_file(transportista)
update_local_file(ruta, True)

<>:68: SyntaxWarning: invalid escape sequence '\/'
<>:68: SyntaxWarning: invalid escape sequence '\/'
C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_16472\1000247876.py:68: SyntaxWarning: invalid escape sequence '\/'
  celda.number_format = 'd\/m\/yy' # Formato visual en Excel
c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Datos nuevos:  97


c:\Users\AYACDA23\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Ultima fila:  11353
✅ Extendiendo tabla 'Tabla5' a nuevo rango: A1:AG11449


C:\Users\AYACDA23\AppData\Local\Temp\ipykernel_16472\1000247876.py:68: SyntaxWarning: invalid escape sequence '\/'
  celda.number_format = 'd\/m\/yy' # Formato visual en Excel


ValueError: Table with name Tabla5 already exists